In [1]:
import h5py
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import balanced_accuracy_score
from PIL import Image
from tensorflow.keras.models import Sequential, Model, load_model, model_from_json
from tensorflow.keras.layers import Dense, Dropout, Input, Activation, Conv2D, Conv1D, MaxPooling2D, Flatten
from tensorflow.keras.optimizers import RMSprop, SGD, Adagrad, Adadelta, Adam
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.applications import ResNet50, ResNet101, ResNet152
from tensorflow.keras import metrics
from tensorflow.keras import losses

# from collections import Counter
# counter_object = Counter(labels_list)
# keys = counter_object.keys()
# print(keys, counter_object.values())
# len(labels_list)

def preprocess_dataset(path_to_h5_file, name):
    samples = h5py.File(path_to_h5_file, 'r')[name + '_spectros']
    labels = h5py.File(path_to_h5_file, 'r')[name + '_labels']

    samples_array = np.zeros(shape=len(samples), dtype=object)
    labels_list = []

    for sample in range(len(samples)):
        samples_array[sample] = samples[str(sample)][()]
        labels_list.append(labels[str(sample)][()].decode('utf-8'))

    label_encoder = LabelEncoder()
    labels_encoded = label_encoder.fit_transform(labels_list)

    x_image_size = int(np.amin([sample.shape[1] for sample in samples_array]) + np.amax([sample.shape[1] for sample in samples_array]) / 2)

    resized_dataset = samples_array
    for index, sample in enumerate(samples_array):
        resized_dataset[index] = np.asarray(Image.fromarray(sample).resize((int(x_image_size), int(sample.shape[0] / 10))))

    reshaped_dataset = np.zeros((resized_dataset.shape[0], resized_dataset[0].shape[0], resized_dataset[0].shape[1]))
    for index, sample in enumerate(resized_dataset):
        reshaped_dataset[index] = sample

    reshaped_dataset = np.reshape(reshaped_dataset, (reshaped_dataset.shape[0], reshaped_dataset.shape[1], reshaped_dataset.shape[2], 1))

    reshaped_dataset_additional_channels = np.repeat(reshaped_dataset, 3, axis=3)

    x_train, x_test, y_train, y_test = train_test_split(reshaped_dataset_additional_channels, labels_encoded, test_size=0.2, shuffle=True)
    x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, shuffle=True)

    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_val = x_val.astype('float32')

    y_train = keras.utils.to_categorical(y_train, len(label_encoder.classes_))
    y_test = keras.utils.to_categorical(y_test, len(label_encoder.classes_))
    y_val = keras.utils.to_categorical(y_val, len(label_encoder.classes_))

    return x_train, x_test, x_val, y_train, y_test, y_val, len(label_encoder.classes_)

def train_model(x_train, x_test, x_val, y_train, y_test, y_val, n_classes, path, epochs=50, save=True):
    model = Sequential()
    pretrained_model_for_demo= tf.keras.applications.ResNet152(include_top=False,
                       input_shape=x_train.shape[1:],
                       pooling='avg',classes=n_classes,
                       weights='imagenet')

    for each_layer in pretrained_model_for_demo.layers:
      each_layer.trainable=False

    model.add(pretrained_model_for_demo)
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(n_classes, activation='softmax'))
    model.compile(optimizer=Adam(learning_rate=0.001),loss='categorical_crossentropy',metrics=['accuracy'])
    datagen = tf.keras.preprocessing.image.ImageDataGenerator(validation_split=0.2)

    model.fit(datagen.flow(x_train, y_train, batch_size=16, subset='training', shuffle=True),
                                    validation_data=datagen.flow(x_val, y_val, batch_size=16, subset='validation', shuffle=True),
                                    epochs=epochs)

    image_pred = model.predict(x_test)

    y_categorized = []
    y_tested = []
    for row in range(len(image_pred)):
        y_categorized.append(np.argmax(image_pred[row]))
        y_tested.append(np.argmax(y_test[row]))

    y_categorized = np.reshape(y_categorized, newshape=(len(y_categorized), 1))
    y_tested = np.reshape(y_tested, newshape=(len(y_categorized), 1))

    print(f'The balanced accuracy score of the model is: {balanced_accuracy_score(y_tested, y_categorized)}')

    if save:
        model.save(path)

In [2]:
x_train, x_test, x_val, y_train, y_test, y_val, n_classes = preprocess_dataset(path_to_h5_file='E:\iot\data\spectrograms.h5', name='peoples_aug')

In [3]:
train_model(x_train, x_test, x_val, y_train, y_test, y_val, n_classes,
            path='E:\iot\models\\resnet152\\peoples_aug', save=True, epochs=30)

Epoch 1/30
327/327 [==============================] - 40s 77ms/step - loss: 2.3876 - accuracy: 0.2595 - val_loss: 1.9284 - val_accuracy: 0.3926
Epoch 2/30
327/327 [==============================] - 21s 63ms/step - loss: 1.8296 - accuracy: 0.4121 - val_loss: 1.7297 - val_accuracy: 0.4387
Epoch 3/30
327/327 [==============================] - 20s 63ms/step - loss: 1.5907 - accuracy: 0.4890 - val_loss: 1.6690 - val_accuracy: 0.4356
Epoch 4/30
327/327 [==============================] - 21s 63ms/step - loss: 1.3957 - accuracy: 0.5388 - val_loss: 1.5457 - val_accuracy: 0.5061
Epoch 5/30
327/327 [==============================] - 21s 63ms/step - loss: 1.2453 - accuracy: 0.5952 - val_loss: 1.4301 - val_accuracy: 0.5184
Epoch 6/30
327/327 [==============================] - 21s 64ms/step - loss: 1.1043 - accuracy: 0.6387 - val_loss: 1.4084 - val_accuracy: 0.5399
Epoch 7/30
327/327 [==============================] - 21s 65ms/step - loss: 0.9930 - accuracy: 0.6760 - val_loss: 1.3351 - val_accuracy:

INFO:tensorflow:Assets written to: E:\iot\models\resnet152\peoples_aug\assets


INFO:tensorflow:Assets written to: E:\iot\models\resnet152\peoples_aug\assets
